In [ ]:
! pip install chromadb langchain pypdf tqdm umap-learn sentence_transformers plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.5 MB/s et

In [1]:
from helper_utils import load_chroma, word_wrap
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()

chroma_collection = load_chroma(filename='/content/microsoft_annual_report_2022.pdf', collection_name='microsoft_annual_report_2022', embedding_function=embedding_function)
chroma_collection.count()

ModuleNotFoundError: No module named 'helper_utils'

In [ ]:
chroma_collection.get(include = ['embeddings'])['embeddings'][0]

[-0.05130083113908768,
 -0.011773261241614819,
 0.04940269514918327,
 -0.07973458617925644,
 0.014822014607489109,
 -0.017085716128349304,
 -0.03712102398276329,
 0.010243202559649944,
 0.029443364590406418,
 -0.011551033705472946,
 -0.029305987060070038,
 0.059875957667827606,
 0.041933659464120865,
 0.0009110342944040895,
 -0.0002481859701219946,
 0.055705588310956955,
 -0.0950448140501976,
 -0.11239242553710938,
 -0.04467061161994934,
 -0.0028009586967527866,
 -0.024321073666214943,
 -0.015662461519241333,
 -0.049685392528772354,
 0.01725567691028118,
 -0.04192959517240524,
 0.02931787073612213,
 -0.021453723311424255,
 -0.07143810391426086,
 -0.03475288674235344,
 0.021638227626681328,
 -0.015290459617972374,
 0.05068206414580345,
 0.029244007542729378,
 0.04425171762704849,
 0.03545273095369339,
 0.022688910365104675,
 0.06838448345661163,
 -0.020329995080828667,
 0.019099781289696693,
 -0.09923567622900009,
 -0.011153696104884148,
 -0.12798891961574554,
 -0.035531435161828995,
 0

In [ ]:
chroma_collection.get(include = ['documents'])['documents'][0]

'1 dear shareholders, colleagues, customers, and partners : we are living through a period of historic economic, societal, and geopolitical change. the world in 2022 looks nothing like the world in 2019. as i write this, inflation is at a 40 - year high, supply chains are stretched, and the war in ukraine is ongoing. at the same time, we are entering a technological era with the potential to power awesome advancements across every sector of our economy and society. as the world ’ s largest software company, this places us at a historic intersection of opportunity and responsibility to the world around us. our mission to empower every person and every organization on the planet to achieve more has never been more urgent or more necessary. for all the uncertainty in the world, one thing is clear : people and organizations in every industry are increasingly looking to digital technology to overcome today ’ s challenges and emerge stronger. and no'

In [ ]:
import umap.umap_ as umap
import numpy as np
from tqdm import tqdm

embeddings = chroma_collection.get(include=['embeddings'])['embeddings']
documents = chroma_collection.get(include=['documents'])['documents']
umap_transform = umap.UMAP(n_components=3, random_state=0, transform_seed=0).fit(embeddings) #3D

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [ ]:
def project_embeddings(embeddings, umap_transform):
    umap_embeddings = np.empty((len(embeddings),3)) #fits 3D
    for i, embedding in enumerate(tqdm(embeddings)):
        umap_embeddings[i] = umap_transform.transform([embedding])
    return umap_embeddings

In [ ]:
projected_dataset_embeddings = project_embeddings(embeddings, umap_transform)

#save projected_dataset_embeddings
np.save('projected_dataset_embeddings.npy', projected_dataset_embeddings)

 22%|██▏       | 78/349 [01:20<05:09,  1.14s/it]

In [ ]:
def truncate_text(text, max_words=20):
    """
    Truncates the text to a maximum of max_words.

    :param text: The original text.
    :param max_words: The maximum number of words to include in the truncated text.
    :return: A truncated version of the text.
    """
    words = text.split()  # Split the text into words
    if len(words) > max_words:
        return ' '.join(words[:max_words]) + '...'  # Join and return the first max_words words
    return text  # Return the original text if it's short enough

In [ ]:
# 4*349 dimension array, column0 is doc and column1&2&3 are projected embeddings
projected_embeddings_docs = np.empty((len(embeddings),4), dtype=object)

for i in range(len(embeddings)):
    projected_embeddings_docs[i] = [truncate_text(documents[i]), projected_dataset_embeddings[i][0], projected_dataset_embeddings[i][1], projected_dataset_embeddings[i][2]]

np.save('projected_embeddings_docs_3d.npy', projected_embeddings_docs)

In [ ]:
import numpy as np

projected_embeddings_docs_3d = np.load('/content/projected_embeddings_docs_3d.npy', allow_pickle=True)

In [ ]:
import plotly.express as px
import pandas as pd

#content 是每个点的标签或句子
df = pd.DataFrame(projected_embeddings_docs_3d, columns=['content', 'x', 'y', 'z'])

fig = px.scatter_3d(df, x='x', y='y', z='z', hover_data=['content'])

fig.update_traces(marker=dict(size=2),
                  selector=dict(mode='markers'))

fig.update_layout(title='Projected Embeddings',
                  scene=dict(
                      xaxis_visible=False, yaxis_visible=False, zaxis_visible=False,
                      xaxis_showgrid=False, yaxis_showgrid=False, zaxis_showgrid=False))
fig.show()

try one retrieval

In [ ]:
query = "What is the total revenue?" #original query

results = chroma_collection.query(query_texts=query, n_results=5, include=['documents', 'embeddings'])

retrieved_documents = results['documents'][0]

for document in results['documents'][0]:
    print(word_wrap(document))
    print('')

revenue, classified by significant product and service offerings, was
as follows : ( in millions ) year ended june 30, 2022 2021 2020 server
products and cloud services $ 67, 321 $ 52, 589 $ 41, 379 office
products and cloud services 44, 862 39, 872 35, 316 windows 24, 761 22,
488 21, 510 gaming 16, 230 15, 370 11, 575 linkedin 13, 816 10, 289 8,
077 search and news advertising 11, 591 9, 267 8, 524 enterprise
services 7, 407 6, 943 6, 409 devices 6, 991 6, 791 6, 457 other 5, 291
4, 479 3, 768 total $ 198, 270 $ 168, 088 $ 143, 015 we have recast
certain previously reported amounts in the table above to conform to
the way we internally manage and monitor our business.

74 note 13 — unearned revenue unearned revenue by segment was as
follows : ( in millions ) june 30, 2022 2021 productivity and business
processes $ 24, 558 $ 22, 120 intelligent cloud 19, 371 17, 710 more
personal computing 4, 479 4, 311 total $ 48, 408 $ 44, 141 changes in
unearned revenue were as follows : ( in millio

In [ ]:
#project embeddings

query_embedding = embedding_function([query])[0]
retrieved_embeddings = results['embeddings'][0] #use this format to get results

projected_query_embedding = project_embeddings([query_embedding], umap_transform)
projected_retrieved_embeddings = project_embeddings(retrieved_embeddings, umap_transform)

100%|██████████| 5/5 [00:06<00:00,  1.26s/it]


In [ ]:
embedding_function([query])[0]

[0.04003918543457985,
 -0.005881161894649267,
 -0.041435644030570984,
 -0.030712557956576347,
 -0.07780002057552338,
 -0.02132018841803074,
 0.04645408317446709,
 0.07020603865385056,
 0.01431006845086813,
 0.012506755068898201,
 -0.028019102290272713,
 -0.026338830590248108,
 -0.020589269697666168,
 -2.3241071176016703e-05,
 -0.08787356317043304,
 0.014907562173902988,
 0.018995661288499832,
 -0.04644031450152397,
 0.05182135850191116,
 -0.017277147620916367,
 0.06887494772672653,
 0.014397997409105301,
 -0.0038656198885291815,
 -0.01959996111690998,
 0.09688466787338257,
 0.01618560217320919,
 -0.048593729734420776,
 0.00933114718645811,
 0.013290084898471832,
 -0.04353189840912819,
 -0.05846427008509636,
 0.016406789422035217,
 0.026699794456362724,
 0.013670513406395912,
 -0.008813678286969662,
 -0.019881494343280792,
 -0.02635054662823677,
 0.008396470919251442,
 0.032966498285532,
 0.03842230513691902,
 -0.0034974324516952038,
 -0.02808731608092785,
 -0.09632141143083572,
 -0.053

In [ ]:
type(embedding_function)

chromadb.utils.embedding_functions.SentenceTransformerEmbeddingFunction

In [ ]:
embedding_function([query]) #list inside a list

[[0.04003918543457985,
  -0.005881161894649267,
  -0.041435644030570984,
  -0.030712557956576347,
  -0.07780002057552338,
  -0.02132018841803074,
  0.04645408317446709,
  0.07020603865385056,
  0.01431006845086813,
  0.012506755068898201,
  -0.028019102290272713,
  -0.026338830590248108,
  -0.020589269697666168,
  -2.3241071176016703e-05,
  -0.08787356317043304,
  0.014907562173902988,
  0.018995661288499832,
  -0.04644031450152397,
  0.05182135850191116,
  -0.017277147620916367,
  0.06887494772672653,
  0.014397997409105301,
  -0.0038656198885291815,
  -0.01959996111690998,
  0.09688466787338257,
  0.01618560217320919,
  -0.048593729734420776,
  0.00933114718645811,
  0.013290084898471832,
  -0.04353189840912819,
  -0.05846427008509636,
  0.016406789422035217,
  0.026699794456362724,
  0.013670513406395912,
  -0.008813678286969662,
  -0.019881494343280792,
  -0.02635054662823677,
  0.008396470919251442,
  0.032966498285532,
  0.03842230513691902,
  -0.0034974324516952038,
  -0.0280873

In [ ]:
type(embedding_function([query]))

list

In [ ]:
import plotly.graph_objs as go
import pandas as pd

# 假设 df 是你的 DataFrame，包含了所有文档的嵌入以及它们的 'content', 'x', 'y', 'z' 值

# 创建数据集的散点图
trace_dataset = go.Scatter3d(
    x=df['x'],
    y=df['y'],
    z=df['z'],
    mode='markers',
    marker=dict(
        size=2,
        color='gray',  # 数据集点的颜色
    ),
    name='Dataset'
)

# 创建查询嵌入的散点图
trace_query = go.Scatter3d(
    x=[projected_query_embedding[0][0]],
    y=[projected_query_embedding[0][1]],
    z=[projected_query_embedding[0][2]],
    mode='markers',
    marker=dict(
        size=5,
        color='red',  # 查询点的颜色
    ),
    name='Query'
)

# 创建检索到的文档的散点图
trace_retrieved = go.Scatter3d(
    x=projected_retrieved_embeddings[:, 0],
    y=projected_retrieved_embeddings[:, 1],
    z=projected_retrieved_embeddings[:, 2],
    mode='markers',
    marker=dict(
        size=5,
        color='green',  # 检索文档点的颜色
        symbol='circle-open'  # 使用空心圆表示检索到的文档
    ),
    name='Retrieved'
)

# 定义布局
layout = go.Layout(
    title='Query and Retrieved Documents in Embedding Space',
    margin=dict(l=0, r=0, b=0, t=0),
    scene=dict(
              xaxis_visible=False, yaxis_visible=False, zaxis_visible=False,
              xaxis_showgrid=False, yaxis_showgrid=False, zaxis_showgrid=False
        )
    )

# 创建图形并显示
fig = go.Figure(data=[trace_dataset, trace_query, trace_retrieved], layout=layout)
fig.show()
